Bu örnekte, güç analizinin bir mikroişlemci üzerinde yapılan işlemleri nasıl söyleyebileceğini, yapılan işlemler ile güç izinin nasıl değiştiğini göreceğiz. Cihazlarda çeşitli güvenlik önlemlerini kırmak için gerekli güç izi profilini incelemeyi bu işlem ile temel olarak öğreneceğiz. 

Test yapmak için hedef uygulamaya özel program kodumuzu yazmalıyız. Bunun için öncelikle kullandığımız platformu ve test edeceğimiz platformu yazdığımız programda tanıtmalıyız.
SCOPETYPE değişkenine kullandığımız Chipwhisperer kartını tanımlıyoruz. CWLite ve CWPro için OPENADC, CWNano için CWNANO tanımlamasını yapıyoruz.
PLATFORM değişkenine test edeceğimiz kartı tanımlıyoruz. API bize temelde 2 ana platformu hazır halde sunuyor. AVR için CWLITEXMEGA, ARM için CWLITEARM platformlarını kullanabiliyoruz. PLATFORM değişkenini tanıtmamızın nedeni yaptığımız uygulamanın mikrodenetleyicisine uygun derleyici kullanmak ve ona uygun işlemleri yapmaktır. API, AVR için avr-gcc --version, ARM için arm-none-eabi-gcc --version kullanmaktadır. Bu, PLATFORM tanımlamasında kullandığımız tanımlamanın (AVR için CWLITEXMEGA, ARM için CWLITEARM) içinde gömülüdür.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

Jupyter Notebook'ta çalıştığımız için buradaki kod bloğumuza %%bash komutu ile Linux bash ekranını çağırıyoruz. Böylelikle sistem içerisindeki dosya ve dizinlere komutlarla erişim sağlayabiliriz. 
Test edeceğimiz sistem için bir firmware kurulumu yapacağız.
cd komutu ile test uygulaması için gerekli firmware'lerin bağlı olduğu dizine geçiyoruz.
mkdir ile firmware'imize dizin oluşturuyoruz ve simpleserial-base dizinindeki her şeyi buraya kopyalıyoruz. Burada firmware olarak API tarafından sunulan simpleserial-base kullanacağız.
cd komutu ile firmware dizinimize giriyoruz.

"SimpleSerial, ChipWhisperer projesinin neredeyse tamamı için kullanılan iletişim protokolüdür. Çoğu sistemde kolayca uygulanabilen çok basit bir seri iletişim protokolüdür. Asenkron, 38400 baud, 8-N-1 kullanarak iletişim kurar."

In [ ]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-base-lab2 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab2

Bu örnekte kripto kullanmayacağımız için CRYPTO_TARGET değişkenini NONE olarak tanımladık. API'nin sunduğu parametrelerden birkaçı: avrcryptolib, mbedtls, micro-ecc, secAES-ATmega8515, tiny-AES128-C

In [ ]:
CRYPTO_TARGET = "NONE"

Firmware dizinimize girerek belirttiğimiz PLATFORM ve CRYPTO_TARGET parametreleriyle make komutunu kullanarak derleme işlemini yapıyoruz.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

Yüklediğimiz yazılım kayda değer bir işlem gerçekleştirmeyen (basit giriş çıkış işlemleri) programdır.
Yüklediğimiz yazılımın güç izini incelemeden önce programın kodunu açarak trigger'ın aktif olduğu high olduğu komut ile low olduğu komut arasına basit bir for döngüsü ekleyeceğiz. Güç izinin, döngünün ne kadar sürdüğüne bağlı olarak nasıl değiştiğine bakacağız.

Değişiklik yaptığımız için tekrar derliyoruz.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

Hedef kart ile bağlantıyı kuruyoruz.

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Varsayılan olarak, kapsam ihtiyaç duyduğumuzdan çok daha fazla iz yakalayacaktır, bu nedenle bunu sample'ı 1000'e indireceğiz. fw_path değişkeni ile derlenmiş kodumuzun yolunu tutuyoruz.

In [ ]:
scope.adc.samples = 1000
fw_path = '../hardware/victims/firmware/simpleserial-base-lab2/simpleserial-base-{}.hex'.format(PLATFORM)

Hedef kartın yazılımını yüklüyoruz.

In [ ]:
cw.program_target(scope, prog, fw_path)

Bu örnekte hedefin kapsam özellikleri ile ilgilenmeyeceğiz. İlgilenecek olursak aşağıdaki komut ile hedef cihazımız ile ilgili bilgilere erişebiliriz.

In [ ]:
print(scope)

Hazırladığımız programın güç izini aşağıdaki kodla inceliyoruz. Burada programa eklediğimiz for dönügüsü ile gecikme oluşturduğumuz için güç izini incelediğimizde başlangıçta 5 tane tepe görüyoruz. Bu for döngümüzün 5 kere tekrar etmesiyle oluşan gecikmedir.

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
ktp = cw.ktp.Basic()
key, text = ktp.next()  # Bir key, text çiftinin elle oluşturulması burada değiştirilebilir

trace = cw.capture_trace(scope, target, text)
hv.Curve(trace.wave).opts(width=600, height=400)

Grafikte döngüyü doğrudan görebiliyoruz. Döngünün bizim müdahalemiz olduğundan emin olmak için kodu ilk haline getirip güç izini tekrar inceleyelim.

Değiştirilmiş kodu tekrar derleyelim.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

Tekrar programlayalım

In [ ]:
cw.program_target(scope, prog, fw_path)

Yeni izi yakalayım ve inceleyelim.

In [ ]:
trace2 = cw.capture_trace(scope, target, text)
hv.Curve(trace2.wave).opts(width=600, height=600)

Chipwhisperer ve test kartımızla bağlantımızı kesiyoruz.

In [ ]:
scope.dis()
target.dis()